In [3]:
!conda install mlxtend
!pip install mlxtend
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
from statsmodels.api import qqplot,add_constant
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from statsmodels.tools.eval_measures import rmse
%matplotlib inline
from sklearn.linear_model import Ridge,Lasso,ElasticNet
from sklearn.model_selection import train_test_split
import matplotlib
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import graphviz
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import warnings
from sklearn.decomposition import PCA
warnings.simplefilter('ignore')
matplotlib.rc('font',family = 'AppleGothic')
matplotlib.rc('axes',unicode_minus=False)
import statsmodels.api as sm
from datetime import datetime
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.api import Logit
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score,roc_curve,auc
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.cluster import KMeans, DBSCAN
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - mlxtend

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


     |████████████████████████████████| 1.3 MB 1.6 MB/s eta 0:00:01


# 연관규칙

== 장바구니 분석
- 평가기준 : 지지도, 신뢰도, 향상도
* 순차분석 : 거래/ 사건의 순서를 고려한 분석
         - 지지도
         
1) 지지도(support)
 - 전체 거래수에서 a,b를 동시에 구매한 비율[0,1]
 - (a,b가 동시에 포함된 거래 수)/전체 거래 수
 $$ P(A \cap B) = (A \cap B)/(Total)$$
 
2) 신뢰도(Confidence)
 - A거래(수)에서 A와B를 동시에 구매한 비율(조건부확률). 범위[0.1]
 - (A와B가 동시에 포함된 거래 수)/(A거래 수)
  $$ P(B|A) = P(A \cap B)/P(A) $$
  
3) 향상도(Lift)
- 전체거래(수)에서 B만 구매한 비율 대비 A를 구매한 경우 B를 동시에 구매한 비율
- (A거래 중 B가 포함 된 구매비율)/(전체 거래 중 B를 구매한 비율)
 $$P(B|A)/P(B)=P(A \cap B)/[P(A)*P(B)]$$

In [4]:
df_raw=[["우유","라면"],["라면"],["아이스크림","우유"],["과자","라면"],["아이스크림"]]
df_raw

[['우유', '라면'], ['라면'], ['아이스크림', '우유'], ['과자', '라면'], ['아이스크림']]

## (1) 데이터 구성하기

In [5]:
enc = TransactionEncoder()
df_raw_enc=enc.fit_transform(X=df_raw)
df_asso=pd.DataFrame(df_raw_enc,columns=enc.columns_)
df_asso.head()

,과자,라면,아이스크림,우유
0,False,True,False,True
1,False,True,False,False
2,False,False,True,True
3,True,True,False,False
4,False,False,True,False


## (2) 라이브러리 불러오기

In [14]:
v_min_sup=0.2
df_freq=apriori(df_asso,min_support=v_min_sup,use_colnames=True)
df_freq.round(3)

,support,itemsets
0,0.2,(과자)
1,0.6,(라면)
2,0.4,(아이스크림)
3,0.4,(우유)
4,0.2,"(과자, 라면)"
5,0.2,"(우유, 라면)"
6,0.2,"(우유, 아이스크림)"


## (3) 지지도,신뢰도,향상도 산출

In [15]:
df_asso_rule=association_rules(df_freq,metric="confidence",min_threshold=0.5)
df_asso_rule.round(3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(과자),(라면),0.2,0.6,0.2,1.0,1.667,0.08,inf
1,(우유),(라면),0.4,0.6,0.2,0.5,0.833,-0.04,0.8
2,(우유),(아이스크림),0.4,0.4,0.2,0.5,1.250,0.04,1.2
3,(아이스크림),(우유),0.4,0.4,0.2,0.5,1.250,0.04,1.2
